In [25]:
# Submission - 1
# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
import glob
import statistics
import os
import math
import datetime
# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

# Specify the file path
directory_path = 'C:\\Abhishek\\IIITD\\Academics\\CSE\\CSE558_DSc\\Project\\PRSA_Data_20130301-20170228'
directory_path2 = 'C:\\Abhishek\\IIITD\\Academics\\CSE\\CSE558_DSc\\Project'


First we combine all the csvs into one, keeping a column for the weather station name separately

In [2]:
# Get a list of all files in the directory
file_list = os.listdir(directory_path)

# Combined dataframe
com_df = []

# Iterate through the files in the directory
for filename in file_list:
    # Construct the full file path
    file_path = os.path.join(directory_path, filename)
    
    # Check if the current item is a file
    if os.path.isfile(file_path):
        "Reading", filename
        df = pd.read_csv(file_path, index_col=None, header=0) # Converting the file to a dataframe
        com_df.append(df)
data = pd.concat(com_df)
data.to_csv('output.csv', index=False)

Then for the sake of uniformity we rename the columns so it remains the same for everyone

In [3]:
data.drop(["No"], axis=1, inplace=True)
data.rename(columns = {'year': 'Year', 'month': 'Month', 'day': "Day", 'hour': 'Hour', 'pm2.5': 'PM2.5', 'DEWP': 'DewP', 'TEMP': 'Temp', 'PRES': 'Press', 'RAIN': 'Rain','wd': 'WinDir', 'WSPM': 'WinSpeed', 'station': 'Station'}, inplace = True)

data.to_csv('dataset.csv', index=False)

We also need to deal with the null and missing values present in this dataset

In [ ]:
data.isnull().sum()

This will be done as follows :
1. For continuous attributes like CO levels etc., collate data for each month and simply replace NA with the average value.
2. For categorical attributes like wind direction, collate the data from the entire dataset and replace with the mode value.

In [ ]:
directory_path2 = 'C:\\Abhishek\\IIITD\\Academics\\CSE\\CSE558_DSc\\Project'
df = pd.read_csv(os.path.join(directory_path2, 'dataset.csv'), index_col=None)
SO2,counts = {}, df['Month'].value_counts()
for i in range(12):
    SO2[i+1] = 0

for index, row in df.iterrows():
    if not row['SO2'] == ' ' and not math.isnan(float(row['SO2'])):
        SO2[row['Month']] += (float(row['SO2'])/counts[row['Month']]) 
average_SO2 = np.array([round(num, 6) for num in SO2.values()])

# Define the columns for which you want to calculate averages
columns_to_average = ['PM2.5','PM10','NO2','CO','O3','Temp','Press','DewP','Rain','WinSpeed'] # Removing Wind direction since we want to use mode for that

# Define the columns with missing values
columns_with_missing = ['WinDir']

# Group the DataFrame by 'month' and calculate the averages for the specified columns
averages_by_month = df.groupby('Month')[columns_to_average].mean()

# Adding the SO2 average as well
averages_by_month['SO2'] = average_SO2

# Group the DataFrame by 'month' and calculate the mode for the categorical column
modes_by_month = df.groupby('Month')[['WinDir']].agg(lambda x: x.mode()[0] if not x.mode().empty else None)

# Print the resulting DataFrame
print(averages_by_month)
# print(modes_by_month)

Now we actually replace the values based on the averages calculated

In [ ]:
# Iterate through the DataFrame and replace null or missing values with the corresponding averages

for index, row in df.iterrows():
    for column in df.columns:
        if pd.isnull(df.at[index, column]):
            if column == 'WinDir':
                df.at[index, column] = modes_by_month.loc[df.at[index, 'Month'], 'WinDir']
            else:
                df.at[index, column] = averages_by_month.loc[df.at[index, 'Month'], column]

print(df[:10])
df.to_csv('Non_Missing_Dataset.csv', index=False)

Finally we add Day and Week as attributes as it gives a more clear sense of what part of the year we are in. Pertaining to seasons etc, which will likely play an important role in weather and pollution statistics.

In [ ]:
def get_week_and_day(month, day,year):
    if month < 1 or month > 12 or day < 1 or day > 31:
        return None  # Invalid input
    
    # Create a date object with the provided month and day
    date_obj = datetime.date(year, month, day)
    
    # Calculate the ISO week number (1-52/53) and weekday (1-7)
    week_number = date_obj.isocalendar()[1]
    day_of_week = date_obj.isocalendar()[2]
    
    return week_number, day_of_week

df2 = df.copy()

weeks = []
days = []
for i in range(len(df)):
    # Calculate the week and day number based of Month, Year and Day details
    week, day = get_week_and_day(df2['Month'][i], df2['Day'][i], df2['Year'][i])
    weeks.append(week)
    days.append(day)

# Set these as new columns
df2['Week'] = weeks
df2['DayOfWeek'] = days

# Final Dataset obtained
df2.to_csv('Final_Dataset.csv', index=False)
